In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# importing required packages
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from keras.utils.vis_utils import plot_model
from IPython import display
import pandas as pd

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data2.txt",header=None)
index = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/index.txt",header=None)
data2 = data2.to_numpy()
index = index.to_numpy()

In [ ]:
print(data.shape)
print(index.shape)
train_dataset = np.reshape(data,(-1,28,28,1))
print(train_dataset.shape)

In [ ]:
print(train_dataset[2].shape)
train_imgs = train_dataset

In [ ]:
N_Imag = 49920
Batch_Size = 12
noise_d = 100
train_set = tf.data.Dataset.from_tensor_slices(train_imgs).shuffle(N_Imag).batch(Batch_Size)

In [ ]:
# Architecture for Generator

def generator_model():
    model = tf.keras.Sequential()
    # First Layer (from noise 100x1 to 7*7*256)
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(noise_d,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) 

    # Second Layer (1st Deconvolution)
    model.add(layers.Conv2DTranspose(128, (5,5), strides=(1,1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    # Thirds Layer (2nd Deconvolution)
    model.add(layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    # Forth Layer (3rd Deconvolution)
    model.add(layers.Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

G = generator_model()
# Making Noise !!
noise = tf.random.normal([1, 100])
# Feeding the noise to the generator
generated_img = G(noise, training=False)
# Plotting the (not-so-fancy) output 
plt.imshow(generated_img[0,:,:,0], cmap='gray')

In [ ]:
G.summary()

In [ ]:
plot_model(G, to_file='model_gen.png', show_shapes=True, show_layer_names=True)

In [ ]:
def discriminator_model():
    model = tf.keras.Sequential()
    # First 2D Conv Layer
    model.add(layers.Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=[28,28,1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    # Second 2D Conv Layer
    model.add(layers.Conv2D(128,(5,5), strides=(2,2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    # Flatten the output of the second layer
    model.add(layers.Flatten())
    # Dense layer for scalar output
    model.add(layers.Dense(1))

    return model

D = discriminator_model()

In [ ]:
print(D.summary())

In [ ]:
plot_model(D, to_file='model_dis.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Defining Binary-Cross-Entropy loss function instance 
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# Discriminator Loss Function
def discriminator_loss(true_output, fake_output):
    T = cross_entropy(tf.ones_like(true_output), true_output)
    F = cross_entropy(tf.zeros_like(fake_output), fake_output)
    D_loss = F + T
    return D_loss
# Generator Loss Function
def generator_loss(fake_output):
    F = cross_entropy(tf.ones_like(fake_output), fake_output)
    return F

In [ ]:
G_Opt = tf.keras.optimizers.Adam(1e-4)
D_Opt = tf.keras.optimizers.Adam(1e-4)
Epochs = 50 

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([Batch_Size, noise_d])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Making a Noise generated image through the Generator
        generated_img = G(noise, training=True) 
        # True and Fake outputs from Discriminator
        T_out = D(images, training=True)
        F_out = D(generated_img, training=True)
        # Generator Loss
        G_Loss = generator_loss(F_out)
        # Discriminator Loss
        D_Loss = discriminator_loss(T_out, F_out)
    # Gradient Computation
    grad_gen = gen_tape.gradient(G_Loss, G.trainable_variables)
    grad_disc = disc_tape.gradient(D_Loss, D.trainable_variables)
    # Updating the weights
    G_Opt.apply_gradients(zip(grad_gen, G.trainable_variables))
    D_Opt.apply_gradients(zip(grad_disc, D.trainable_variables))
    # Returning the Losses
    return tf.cast(G_Loss, tf.float32),tf.cast(D_Loss, tf.float32)

In [ ]:
def train(dataset, epochs):

    G_Loss_History = []
    D_Loss_History = []
    for epoch in range(epochs):
        start = time.time()
        G_Losses = []
        D_Losses = []
        for img_batch in dataset:
            G_Loss, D_Loss = train_step(img_batch)
            # Saving the Loss values in an array for the entire dataset
            G_Losses.append(G_Loss)
            D_Losses.append(D_Loss)
            
        # History of all Loss Function Arrays
        G_Loss_History.append(G_Losses)
        D_Loss_History.append(D_Losses)

    # Returning Errors
    return G_Loss_History, D_Loss_History

In [ ]:
G_Loss_History, D_Loss_History = train(train_set, Epochs)

In [ ]:
G_Loss = []
D_Loss = []
for i in range(Epochs):
  G_Loss_i = G_Loss_History[i]
  Mean = sum(G_Loss_i)/len(G_Loss_i)
  G_Loss.append(Mean.numpy())
  D_Loss_i = D_Loss_History[i]
  Mean = sum(D_Loss_i)/len(D_Loss_i)
  D_Loss.append(Mean.numpy())

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 5))
ax1.plot(G_Loss, 'b-')
ax1.set_ylabel('Generator Loss', color='b')
ax1.set_xlabel('Epochs',color='k')
ax2 = ax1.twinx()
ax2.plot(D_Loss, 'r-')
ax2.set_ylabel('Discriminator Loss', color='r')
plt.title('Batch Size = 32')
plt.show()

In [ ]:
seed = tf.random.normal([Batch_Size, noise_d])

In [ ]:
predictions = G(seed, training=False)
predictions = predictions.numpy()
predictions = np.reshape(predictions,(28,28))
print(predictions.shape)

In [ ]:
df = pd.DataFrame(data=predictions)
df.to_csv("./GAN5_out.csv", sep=',',index=False)